In [1]:
import os
import sys
import numpy as np
from sklearn.metrics import hamming_loss, accuracy_score, precision_score, recall_score, f1_score

In [2]:
NPY_FILE_PATH = "../npy_files/13Jun2023_without_multitask"

filenames = os.listdir(path=NPY_FILE_PATH)
filenames = sorted(filenames, key=lambda x: x.split('_')[-1])
filenames = list(set([filename.split('_')[2] for filename in filenames]))

In [3]:
def hamming_score(y_true, y_pred):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    
    return np.mean(acc_list)

In [7]:
filename = filenames[0]
filenames

['expert-rain-246.npy',
 'toasty-disco-254.npy',
 'vital-universe-262.npy',
 'twilight-snowflake-263.npy',
 'fresh-jazz-256.npy',
 'upbeat-shape-249.npy',
 'charmed-durian-272.npy',
 'gallant-breeze-263.npy',
 'swift-haze-246.npy',
 'worthy-firefly-260.npy',
 'devout-blaze-255.npy',
 'morning-waterfall-256.npy',
 'playful-jazz-265.npy',
 'fallen-yogurt-260.npy',
 'pious-flower-245.npy',
 'flowing-waterfall-256.npy',
 'pious-puddle-249.npy',
 'fanciful-puddle-265.npy',
 'ruby-firefly-262.npy',
 'olive-shape-262.npy',
 'easy-breeze-252.npy',
 'distinctive-wood-259.npy',
 'quiet-snowflake-265.npy',
 'desert-star-249.npy',
 'driven-wildflower-253.npy',
 'radiant-night-265.npy',
 'eager-tree-265.npy']

In [8]:
eval_labels_data = np.load(f'{NPY_FILE_PATH}/eval_labels_{filename}', allow_pickle=True)

In [14]:
eval_labels_data[0][1][0]

[0.0, 0.0, 0.0, 1.0, 0.0]

In [16]:
for filename in filenames:
        run_name = filename.split('.')[0]
        print(f"The metrics for {run_name} run are:")
        labels_data = np.load(f'{NPY_FILE_PATH}/test_labels_{filename}', allow_pickle=True)
        preds_data = np.load(f'{NPY_FILE_PATH}/test_preds_{filename}', allow_pickle=True)
        eval_labels_data = np.load(f'{NPY_FILE_PATH}/eval_labels_{filename}', allow_pickle=True)
        eval_preds_data = np.load(f'{NPY_FILE_PATH}/eval_preds_{filename}', allow_pickle=True)
        labels = []
        preds = []
        eval_labels = []
        eval_preds = []
        for i in range(len(labels_data)):
                for j in range(len(labels_data[i][1])):
                        labels.append(labels_data[i][1][j])
                        preds.append(preds_data[i][1][j])
        for i in range(len(eval_labels_data)):
                for j in range(len(eval_labels_data[i][1])):
                        labels.append(eval_labels_data[i][1][j])
                        preds.append(eval_preds_data[i][1][j])
        # for i in range(len(eval_labels_data)):
        # for j in range(len(eval_labels_data[i])):
        #         eval_labels.append(eval_labels_data[i][j][:5])
        #         eval_preds.append(eval_preds_data[i][j][:5])
        labels = np.array(labels)
        preds = np.array(preds)
        eval_labels = np.array(eval_labels)
        eval_preds = np.array(eval_preds)
        print(labels.shape, preds.shape, eval_labels.shape, eval_preds.shape)
        max_hamming_score = 0
        max_thres = 0
        for thresh in np.linspace(0, 1, 11):
                copy_preds = eval_preds.copy()
                copy_preds[copy_preds >= thresh] = 1
                copy_preds[copy_preds < thresh] = 0
                if max_hamming_score < hamming_score(eval_labels, copy_preds):
                        max_hamming_score = hamming_score(eval_labels, copy_preds)
                        max_thres = thresh
                        # max_hamming_loss = hamming_loss(eval_labels, copy_preds)
        copy_preds = preds.copy()
        copy_preds[copy_preds >= max_thres] = 1
        copy_preds[copy_preds < max_thres] = 0                
        
        print("Thresh:", max_thres, "\tHamming Loss:", hamming_loss(copy_preds, labels), "\tHamming Score:", hamming_score(copy_preds, labels))
        preds[preds >= 0.5] = 1
        preds[preds < 0.5] = 0
        
        mapping = {0: "Organisation", 1: "Location", 2: "Individual", 3: "Community", 4: "None"}
        for i in range(0, 5):
                print(f"{mapping[i]} Precision: {precision_score(labels[:, i], preds[:, i])} Recall: {recall_score(labels[:, i], preds[:, i])} F1 Score: {f1_score(labels[:, i], preds[:, i])}")
        print()

The metrics for expert-rain-246 run are:
(888, 5) (888, 5) (0,) (0,)
Thresh: 0 	Hamming Loss: 0.26238738738738737 	Hamming Score: 0.28378378378378377
Organisation Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Location Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Individual Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Community Precision: 0.0 Recall: 0.0 F1 Score: 0.0
None Precision: 0.0 Recall: 0.0 F1 Score: 0.0

The metrics for toasty-disco-254 run are:
(888, 5) (888, 5) (0,) (0,)
Thresh: 0 	Hamming Loss: 0.4632882882882883 	Hamming Score: 0.17802177177177175
Organisation Precision: 0.03125 Recall: 0.0392156862745098 F1 Score: 0.034782608695652174
Location Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Individual Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Community Precision: 0.20588235294117646 Recall: 0.17676767676767677 F1 Score: 0.19021739130434784
None Precision: 0.86 Recall: 0.12684365781710916 F1 Score: 0.2210796915167095

The metrics for vital-universe-262 run are:
(888, 5) (888, 5) (0,) (0,)
Thre

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(

(888, 5) (888, 5) (0,) (0,)
Thresh: 0 	Hamming Loss: 0.5878378378378378 	Hamming Score: 0.10266516516516515
Organisation Precision: 0.05916473317865429 Recall: 1.0 F1 Score: 0.11171960569550932
Location Precision: 0.09264305177111716 Recall: 0.6296296296296297 F1 Score: 0.16152019002375295
Individual Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Community Precision: 0.0 Recall: 0.0 F1 Score: 0.0
None Precision: 0.12280701754385964 Recall: 0.12389380530973451 F1 Score: 0.12334801762114536

The metrics for swift-haze-246 run are:
(888, 5) (888, 5) (0,) (0,)
Thresh: 0 	Hamming Loss: 0.5033783783783784 	Hamming Score: 0.26666666666666666
Organisation Precision: 0.02 Recall: 0.0196078431372549 F1 Score: 0.0198019801980198
Location Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Individual Precision: 0.3055555555555556 Recall: 0.03618421052631579 F1 Score: 0.06470588235294118
Community Precision: 0.24879227053140096 Recall: 0.5202020202020202 F1 Score: 0.3366013071895425
None Precision: 0.56410256410256

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_sc

Thresh: 0 	Hamming Loss: 0.440990990990991 	Hamming Score: 0.20129504504504506
Organisation Precision: 0.02608695652173913 Recall: 0.17647058823529413 F1 Score: 0.045454545454545456
Location Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Individual Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Community Precision: 0.24081632653061225 Recall: 0.29797979797979796 F1 Score: 0.26636568848758463
None Precision: 0.0 Recall: 0.0 F1 Score: 0.0

The metrics for fanciful-puddle-265 run are:
(888, 5) (888, 5) (0,) (0,)
Thresh: 0 	Hamming Loss: 0.5792792792792792 	Hamming Score: 0.1628003003003003
Organisation Precision: 0.0737410071942446 Recall: 0.803921568627451 F1 Score: 0.13509060955518945
Location Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Individual Precision: 0.2962264150943396 Recall: 0.5164473684210527 F1 Score: 0.37649880095923255
Community Precision: 0.0 Recall: 0.0 F1 Score: 0.0
None Precision: 0.0 Recall: 0.0 F1 Score: 0.0

The metrics for ruby-firefly-262 run are:
(888, 5) (888, 5) (0,) (0,)
Thr

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(

(888, 5) (888, 5) (0,) (0,)
Thresh: 0 	Hamming Loss: 0.5191441441441441 	Hamming Score: 0.2175675675675676
Organisation Precision: 0.06369426751592357 Recall: 0.39215686274509803 F1 Score: 0.10958904109589042
Location Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Individual Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Community Precision: 0.173109243697479 Recall: 0.5202020202020202 F1 Score: 0.2597730138713745
None Precision: 0.0 Recall: 0.0 F1 Score: 0.0

The metrics for radiant-night-265 run are:
(888, 5) (888, 5) (0,) (0,)
Thresh: 0 	Hamming Loss: 0.6738738738738739 	Hamming Score: 0.16659159159159156
Organisation Precision: 0.06837606837606838 Recall: 0.47058823529411764 F1 Score: 0.11940298507462689
Location Precision: 0.02564102564102564 Recall: 0.018518518518518517 F1 Score: 0.02150537634408602
Individual Precision: 0.14285714285714285 Recall: 0.12828947368421054 F1 Score: 0.13518197573656843
Community Precision: 1.0 Recall: 0.005050505050505051 F1 Score: 0.010050251256281409
None Preci

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/mi

In [29]:
preds[preds >= 0.5] = 1
preds[preds < 0.5] = 0

In [30]:
mapping = {0: "Organisation", 1: "Location", 2: "Individual", 3: "Community", 4: "None"}
for i in range(0, 5):
    print(f"{mapping[i]} Precision: {precision_score(labels[:, i], preds[:, i])} Recall: {recall_score(labels[:, i], preds[:, i])} F1 Score: {f1_score(labels[:, i], preds[:, i])}")

Organisation Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Location Precision: 0.0 Recall: 0.0 F1 Score: 0.0
Individual Precision: 0.5288461538461539 Recall: 0.36666666666666664 F1 Score: 0.4330708661417323
Community Precision: 0.36923076923076925 Recall: 0.24242424242424243 F1 Score: 0.29268292682926833
None Precision: 0.5925925925925926 Recall: 0.7783783783783784 F1 Score: 0.6728971962616822


/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
preds

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       ...,
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [13]:
idx = []
for i in range(len(preds)):
    if (preds[i] != labels[i]).any():
        idx.append(i)

In [18]:
import pandas as pd

In [22]:
df = pd.read_csv('data/hate_towards_whom_aug/test.csv')
df = df.iloc[idx]

In [ ]:
[label for label in labels[idx].tolist()]

In [31]:
df['prediction'] = [",".join([mapping[i] for i, _pred in enumerate(pred) if _pred == 1]) for pred in preds[idx].tolist()]

In [34]:
df.tail(50)

,url,category,comment,label,target,prediction
63,https://www.bitchute.com/video/IqAY5vQy4IE4/,news & politics,Now that's how railroads of developed country ...,yes,Location,"Location,Community"
69,https://www.youtube.com/watch?v=VH0rNhUKOto,sports,This what happens when you just inherited mone...,yes,Individual,Community
71,https://www.youtube.com/watch?v=dExrKx7hE6A,health,The truth will shine it’s brightness through t...,yes,Organisation,Individual
74,https://www.youtube.com/watch?v=mLTBW7E6q4w,news & politics,She Mark Zuckerberg dressed in Drag. Don’t be ...,yes,Individual,"Individual,Community"
75,https://www.youtube.com/watch?v=PCVXj4g10j8,sports,aight mate I'm on my way to your house with an...,yes,Individual,Location
77,https://www.youtube.com/watch?v=W6AnCcxsRSk,religion,"This is absolutely shame, Indians giving z plu...",yes,Location,"Individual,Community"
83,https://www.bitchute.com/video/2HGLRJWmq7U/,news & politics,That Asian chinese woman has to go back too.,yes,Individual,Community
84,https://www.youtube.com/watch?v=iNxJf6V6nds,entertainment,Because of westerners,yes,Community,Individual
97,https://www.youtube.com/watch?v=duY9YfA-kuI&ab...,entertainment,John Oliver is living proof that the English c...,yes,Individual,"Individual,Community"
103,https://www.youtube.com/watch?v=mnQBiEw_W1g,news & politics,These opposition states will drive us to Slavery,yes,Organisation,Location
